In [ ]:
!pip install -r "E:\Darsh\DP\Internship&Works\AtliQ Technologies\NER Resume Model\requirements.txt"

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
#nltk.download('word_tokenize')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
import json
import random
import logging
import spacy
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from sklearn.metrics import accuracy_score


LABEL = ["name", "phone_no", "email", "linkedin", "github", "designation",
         "company", "job-duration", "Experience", "degree", "academic-institute", "databases", "tools", "core-skills", "soft-skills", "cloud-platforms", 
         "Front End", "Back End", "Mobile App", "Libraries"]


import json
training_data = []
with open("darsh.patel.jsonl", encoding="utf8") as d:
  for line in d:
    data = json.loads(line)
    text = data["data"]
    entity = data["label"]
    entities = []
    for ent in entity:
      entities.append((ent[0], ent[1], ent[2]))
    training_data.append((text, {"entities" : entities}))
training_data

################### Train Spacy NER.###########

TRAIN_DATA = training_data[:80]
nlp = spacy.blank('en')  # create blank Language class
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
    

for i in LABEL:
  ner.add_label(i)

# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(200):
        print("Statring iteration " + str(itn))
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in TRAIN_DATA:
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.3,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)


Statring iteration 0
{'ner': 8574.036128775599}
Statring iteration 1
{'ner': 5072.394217457038}
Statring iteration 2
{'ner': 5484.0415017472615}
Statring iteration 3
{'ner': 5205.3023747480065}
Statring iteration 4
{'ner': 4460.531768724385}
Statring iteration 5
{'ner': 5187.531909070245}
Statring iteration 6
{'ner': 4283.982748523464}
Statring iteration 7
{'ner': 4536.99903000283}
Statring iteration 8
{'ner': 4246.561919070665}
Statring iteration 9
{'ner': 3792.0102417011367}
Statring iteration 10
{'ner': 4086.7993876077803}
Statring iteration 11
{'ner': 3288.590830162846}
Statring iteration 12
{'ner': 3103.7065372437373}
Statring iteration 13
{'ner': 2960.5185826451907}
Statring iteration 14
{'ner': 2791.3230216636225}
Statring iteration 15
{'ner': 2849.5955305404677}
Statring iteration 16
{'ner': 2615.1747960245466}
Statring iteration 17
{'ner': 2760.296276882593}
Statring iteration 18
{'ner': 2415.499774517762}
Statring iteration 19
{'ner': 2310.9208505017386}
Statring iteration 20

{'ner': 335.092669149573}
Statring iteration 166
{'ner': 389.8304063091026}
Statring iteration 167
{'ner': 343.3935896293014}
Statring iteration 168
{'ner': 424.37354761766056}
Statring iteration 169
{'ner': 447.6785924823229}
Statring iteration 170
{'ner': 366.248300570005}
Statring iteration 171
{'ner': 400.3288232428194}
Statring iteration 172
{'ner': 377.6597640546957}
Statring iteration 173
{'ner': 370.6018590549291}
Statring iteration 174
{'ner': 403.08430935948627}
Statring iteration 175
{'ner': 411.13374391842063}
Statring iteration 176
{'ner': 396.64115181583685}
Statring iteration 177
{'ner': 364.39188545588786}
Statring iteration 178
{'ner': 308.399893133337}
Statring iteration 179
{'ner': 335.43686919232135}
Statring iteration 180
{'ner': 391.4684275491683}
Statring iteration 181
{'ner': 297.5499644480434}
Statring iteration 182
{'ner': 291.7545481463387}
Statring iteration 183
{'ner': 338.8421354633059}
Statring iteration 184
{'ner': 371.506156422751}
Statring iteration 18

In [6]:
#test the model and evaluate it

examples = training_data[80:]
tp=0
tr=0
tf=0

ta=0
c=0        
for text,annot in examples:

    f=open("resume"+str(c)+".txt","w+", encoding="utf8")
    doc_to_test=nlp(text)
    d={}
    for ent in doc_to_test.ents:
        d[ent.label_]=[]
    for ent in doc_to_test.ents:
        d[ent.label_].append(ent.text)

    for i in set(d.keys()):
        f.write("\n\n")
        f.write(i +":"+"\n")
        for j in set(d[i]):
            f.write(j.replace('\n','')+"\n")
    c+=1

nlp.to_disk("my_model")


In [ ]:
import mlflow
import mlflow
import mlflow.spacy

# MLflow Tracking
nlp = spacy.load("my_model")
with mlflow.start_run(run_name='Spacy'):
    mlflow.set_tag('model_flavor', 'spacy')
    mlflow.spacy.log_model(spacy_model=nlp, artifact_path='model')
    #mlflow.log_metric(('accuracy', 0.72))
    my_run_id = mlflow.active_run().info.run_id


# MLflow Models
model_uri = f'runs:/{my_run_id}/model'
nlp2 = mlflow.spacy.load_model(model_uri=model_uri)
# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &")

2021/07/12 11:24:05 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2021/07/12 11:24:06 WARNING mlflow.spacy: Generating only the spacy flavor for the provided spacy model. This means the model can be loaded back via `mlflow.spacy.load_model`, but cannot be 